# Step 1: Setting UP the Environment and Trade Parameters

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import alpaca
from scipy.optimize import brentq
from datetime import datetime, time, timedelta, date
from typing import List, Dict, Tuple, Optional
from dotenv import load_dotenv
from zoneinfo import ZoneInfo
import os

from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import (
    MarketOrderRequest, 
    GetOptionContractsRequest, 
    MarketOrderRequest, 
    OptionLegRequest, 
    ClosePositionRequest,
)

from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.data.requests import StockBarsRequest, StockQuotesRequest, OptionLatestQuoteRequest, OptionBarsRequest, OptionSnapshotRequest, OptionTradesRequest
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderClass,
    OrderType,
    TimeInForce,
    QueryOrderStatus,
    ContractType
)

In [2]:
# keys is stored in config.env file
# - Specify the path to the config.env file
dotenv_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), '.env')
# Load environment variables from the specified environment file (e.g. .env)
load_dotenv(dotenv_path)

# API credentials for Alpaca
API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv('ALPACA_SECRET_KEY')
BASE_URL = None
## We use paper environment for this example
PAPER = True # Please do not modify this. This example is for paper trading only.

# Initialize Alpaca clients
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)
option_historical_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET, url_override=BASE_URL)
stock_data_client = StockHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET)


# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

In [21]:
# Underlying symbol
underlying_symbol = 'SPY'

# Risk free rate for the options greeks and IV calculations
RISK_FREE_RATE = 0.01

# Set the timezone
NY_TZ = ZoneInfo('America/New_York')

# Get current date in US/Eastern timezone
today = datetime.now(NY_TZ).date()
start_date = today - timedelta(days=4)
end_date = today - timedelta(days=1)

# Define delta thresholds
SHORT_PUT_DELTA_RANGE = (-0.60, -0.20)
LONG_PUT_DELTA_RANGE = (-0.40, -0.20)

# Set minimum credit percentage (33%)
MIN_CREDIT_PERCENTAGE = 0.33

# Set stop loss threshold threshold (2 times)
DELTA_STOP_LOSS_THRES = 2.5

# Set target profit and stop-loss levels
TARGET_STOP_LOSS_PERCENTAGE = 0.5


# Getting Historical Stock Bar Data

In [4]:
def get_daily_stock_bars_df(symbol: str, start_date: date, end_date: date) -> pd.DataFrame:
    """
    Return a tidy daily-bar DataFrame for `symbol` between the two dates.
    """
    
    req = StockBarsRequest(
        symbol_or_symbols=symbol,
        timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),
        start=start_date,
        end=end_date,
    )
    resp = stock_data_client.get_stock_bars(req)
    bars = resp.data[symbol]                

    rows = [b.model_dump() for b in bars]

    df = (pd.DataFrame(rows).set_index("timestamp").sort_index())

    return df

In [5]:
stock_bars_data = get_daily_stock_bars_df(symbol='SPY', start_date=start_date, end_date=end_date)
stock_bars_data

# Getting Historical Options Bars Data by Expiration

In [6]:
def calculate_strike_price_range(high_price, low_price, buffer_pct=0.05):
    """
    Calculate the strike price range based on daily high/low with buffer.
    """
    min_strike = low_price * (1 - buffer_pct)
    max_strike = high_price * (1 + buffer_pct)
    
    return min_strike, max_strike

def generate_put_option_symbols(underlying: str, expiration_date: date, min_strike: float, max_strike: float, strike_increment: float = 1) -> List[str]:
    """
    Generate put option symbols for the given parameters.
    
    Args:
        underlying: Underlying symbol (e.g., 'SPY')
        expiration_date: Option expiration date
        min_strike: Minimum strike price
        max_strike: Maximum strike price
        strike_increment: Strike price increment (default 1)
    
    Returns:
        List of option symbols
    """
    option_symbols = []
    
    # Format expiration date as YYMMDD
    exp_str = expiration_date.strftime("%y%m%d")
    
    # Generate strikes in increments
    current_strike = np.ceil(min_strike / strike_increment) * strike_increment
    
    while current_strike <= max_strike:
        # Format strike price as 8-digit integer (multiply by 1000)
        strike_formatted = f"{int(current_strike * 1000):08d}"
        
        # Create option symbol: SPY + YYMMDD + P + 8-digit strike
        option_symbol = f"{underlying}{exp_str}P{strike_formatted}"
        option_symbols.append(option_symbol)
        
        current_strike += strike_increment
    
    return option_symbols

In [7]:
def collect_option_symbols_by_expiration(stock_bars_data, underlying_symbol, buffer_pct):
    """
    Collect option symbols grouped by expiration date based on stock bars data 
    buffer_pct is the buffer percentage for the strike price range

    Returns:
        dict: Dictionary with expiration dates as keys and lists of option symbols as values
    """
    # Collect all symbols
    option_symbols_by_expiration = {}

    for index, row in stock_bars_data.iterrows():
        min_strike, max_strike = calculate_strike_price_range(row['high'], row['low'], buffer_pct=buffer_pct)
        
        # Extract date from the timestamp index for 0DTE (same day expiration)
        expiration_date = index.date()
        # print(expiration_date)

        option_symbols = generate_put_option_symbols(underlying_symbol, expiration_date=expiration_date, min_strike=min_strike, max_strike=max_strike, strike_increment=1)

        # Group symbols by expiration date
        if expiration_date not in option_symbols_by_expiration:
            option_symbols_by_expiration[expiration_date] = []
        
        option_symbols_by_expiration[expiration_date].extend(option_symbols)

    return option_symbols_by_expiration

In [ ]:
option_symbols_by_expiration = collect_option_symbols_by_expiration(stock_bars_data, underlying_symbol, buffer_pct=0.05)
option_symbols_by_expiration

In [12]:
def get_stock_and_option_bars_data(option_symbols_by_expiration: List[str], underlying_symbol, time_interval: int = 10) -> Dict[datetime, List[Dict]]:
    """
    Get option bars data (default 10 minutes time interval) for the specified symbols and date range.
    
    Returns:
        Dictionary with timestamp as key and list of option data dictionaries as value
    """
    option_bars_by_timestamp = {}
    
    for expiration_date, symbols_list in option_symbols_by_expiration.items():

        # Create start and end datetime with specific times and timezone
        start_datetime = datetime(expiration_date.year, expiration_date.month, expiration_date.day, 13, 30)
        end_datetime = datetime(expiration_date.year, expiration_date.month, expiration_date.day, 20, 0)
        print(f"Time range: {start_datetime} to {end_datetime}")

        # Convert market close time to UTC
        market_close_utc = datetime(expiration_date.year, expiration_date.month, expiration_date.day, 16, 0, tzinfo=ZoneInfo("America/New_York")).astimezone(ZoneInfo("UTC"))
        print(f"Market close time in UTC: {market_close_utc}")

        # Get stock bar data for the underlying symbol
        stock_req = StockBarsRequest(
            symbol_or_symbols=underlying_symbol,
            timeframe=TimeFrame(amount=time_interval, unit=TimeFrameUnit.Minute),
            start=start_datetime,
            end=end_datetime
        )
        stock_resp = stock_data_client.get_stock_bars(stock_req)

        # Create a dictionary of stock close prices by timestamp for quick lookup
        stock_close_by_timestamp = {}
        if underlying_symbol in stock_resp.data:
            for stock_bar in stock_resp.data[underlying_symbol]:
                stock_dict = stock_bar.model_dump()
                stock_close_by_timestamp[stock_dict['timestamp']] = stock_dict['close']
            print(f"Retrieved stock data for {underlying_symbol}: {len(stock_resp.data[underlying_symbol])} bars")

        # Get option bar data
        req = OptionBarsRequest(
            symbol_or_symbols=symbols_list,
            timeframe=TimeFrame(amount=time_interval, unit=TimeFrameUnit.Minute),
            start=start_datetime,
            end=end_datetime
        )
        bars_response = option_historical_data_client.get_option_bars(req)
            
       # Extract and format data for each symbol
        for symbol in symbols_list:
            if symbol in bars_response.data:
                bars = bars_response.data[symbol]
                print(f"Retrieved data for {symbol}: {len(bars)} bars")
                
                # Process each bar and organize by timestamp
                strike_price = extract_strike_price_from_symbol(symbol)

                # Process each bar and organize by timestamp
                for bar in bars:
                    bar_dict = bar.model_dump()
                    timestamp = bar_dict['timestamp']

                    # Add strike price to the bar dictionary
                    bar_dict['strike_price'] = strike_price

                    # Add expiration date to the bar dictionary
                    bar_dict['expiry'] = market_close_utc
                    
                    # Add stock close price if available for this timestamp
                    if timestamp in stock_close_by_timestamp:
                        bar_dict['underlying_close'] = stock_close_by_timestamp[timestamp]
                    
                    # Initialize timestamp key if it doesn't exist
                    if timestamp not in option_bars_by_timestamp:
                        option_bars_by_timestamp[timestamp] = []
                    
                    # Add this bar data to the timestamp group
                    option_bars_by_timestamp[timestamp].append(bar_dict)
                    
            else:
                print(f"No data found for {symbol}")

    # Sort timestamps for consistent ordering
    sorted_timestamps = sorted(option_bars_by_timestamp.keys())
    sorted_option_bars_by_timestamp = {ts: option_bars_by_timestamp[ts] for ts in sorted_timestamps}

    
    return sorted_option_bars_by_timestamp


def extract_strike_price_from_symbol(symbol: str) -> float:
    """
    Extract strike price from option symbol.
    Example: 'SPY250616P00571000' -> 571.0
    """
    # Option symbols typically have format: TICKER + YYMMDD + (C/P) + 8-digit strike price
    # The last 8 digits represent strike price * 1000
    try:
        # Extract the last 8 digits and convert to strike price
        strike_str = symbol[-8:]
        strike_price = float(strike_str) / 1000.0
        return strike_price
    except (ValueError, IndexError):
        print(f"Warning: Could not extract strike price from symbol {symbol}")
        return 0.0

In [ ]:
historical_stock_and_option_bars_data = get_stock_and_option_bars_data(option_symbols_by_expiration, underlying_symbol, time_interval=10)
historical_stock_and_option_bars_data

## Calculating Options Greek (Delta) and IV from Historical Option Bars

In [19]:
# Calculate historical option Delta
def calculate_delta_historical(option_price, strike_price, expiry, underlying_price, risk_free_rate, option_type, timestamp):

    # Calculate the time to expiry in years
    T = (expiry - timestamp).total_seconds() / (365 * 24 * 60 * 60)
    # Set minimum T to avoid zero
    T = max(T, 1e-6)

    if T == 1e-6:
        print("Option has expired or is expiring now; setting delta based on intrinsic value.")
        if option_type == 'put':
            return -1.0 if underlying_price < strike_price else 0.0
        else:
            return 1.0 if underlying_price > strike_price else 0.0

    implied_volatility = calculate_implied_volatility(option_price, underlying_price, strike_price, T, risk_free_rate, option_type)
    if implied_volatility is None:
        print("Implied volatility could not be determined, skipping delta calculation.")
        return None

    d1 = (np.log(underlying_price / strike_price) + (risk_free_rate + 0.5 * implied_volatility ** 2) * T) / (implied_volatility * np.sqrt(T))
    delta = norm.cdf(d1) if option_type == 'call' else -norm.cdf(-d1)
    return delta


# Calculate implied volatility
def calculate_implied_volatility(option_price, S, K, T, r, option_type):

    # Define a reasonable range for sigma
    sigma_lower = 1e-6
    sigma_upper = 5.0  # Adjust upper limit if necessary

    # Check if the option is out-of-the-money and price is close to zero
    intrinsic_value = max(0, (S - K) if option_type == 'call' else (K - S))
    if option_price <= intrinsic_value + 1e-6:

        # print("Option price is close to intrinsic value; implied volatility is near zero.") # Uncomment for checking the status

        return 0.0

    # Define the function to find the root
    def option_price_diff(sigma):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        if option_type == 'call':
            price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        elif option_type == 'put':
            price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return price - option_price

    try:
        return brentq(option_price_diff, sigma_lower, sigma_upper)
    except ValueError as e:
        print(f"Failed to find implied volatility: {e}")
        return None

## Finding Short and Long Puts for a 0DTE Bull Put Spread

This is the main algorithmic trading logic that forms the core of our 0DTE bull put spread options strategy. The `find_short_and_long_puts` function implements the systematic approach to identify and select optimal option pairs for bull put spreads. 

**Note**: You can uncomment print functions within `find_short_and_long_puts` to assess the algorithm as well

<b>Algorithm Overview</b>

The function scans through historical market data chronologically to find the first valid option pair that meets our trading criteria:
1. **Delta-based Selection**: 
   - Short Put: Higher delta (closer to ATM) - generates premium income
   - Long Put: Lower delta (further OTM) - provides downside protection

2. **Spread Width Validation**: 
   - Ensures risk/reward ratio stays within acceptable bounds
   - Configurable range (default: $2-$4) balances premium vs. risk

3. **Chronological Priority**: 
   - Selects the first valid pair found in time sequence
   - Mimics real-world trading where timing matters


<b>Algorithm Flow</b>

```json
For each timestamp in historical data (starting from the earliest timestamp):
└── For each option in that timestamp:
    ├── Calculate option delta
    ├── Check if delta fits short put criteria → Store if match
    ├── Check if delta fits long put criteria → Store if match
    └── If both options found:
        ├── Validate spread width
        ├── If valid → Return pair (STOP)
        └── If invalid → Reset and continue search
```

You can uncomment print functions within `find_short_and_long_puts` to assess the algorithm as well

In [61]:
def find_short_and_long_puts(historical_stock_and_option_bars_data, risk_free_rate, short_put_delta_range, long_put_delta_range, spread_width=(2, 4)):
    """
    Identify the short put and long put from the options chain.
    Returns dictionaries containing details of the selected options.
    """
    short_put = None
    long_put = None

    for timestamp, bars_list in historical_stock_and_option_bars_data.items():
        print(f"Analyzing timestamp: {timestamp}")

        for bar_data in bars_list:
            option_symbol = bar_data['symbol']
            # print(f"Option symbol is: {option_symbol}")
    
            # Use the pre-calculated values from bar_data
            underlying_price = bar_data['underlying_close']
            option_price = bar_data['close']  # May use vwap or open price if preferred
            strike_price = bar_data['strike_price']
            expiry = bar_data['expiry']
            timestamp = bar_data['timestamp']

            # Calculate delta
            delta = calculate_delta_historical(
                option_price=option_price,
                strike_price=strike_price,
                expiry=expiry,
                underlying_price=underlying_price,
                risk_free_rate=risk_free_rate,
                option_type='put',
                timestamp=timestamp
            )
            # print(f"Delta for {option_symbol} is: {delta}")
            # Skip this option if delta calculation failed
            if delta is None:
                print(f"Delta calculation failed for {option_symbol} at timestamp: {timestamp}")
                continue
            
            # Check if this option meets short put criteria
            if not short_put and short_put_delta_range[0] <= delta <= short_put_delta_range[1]:
                short_put = create_option_dict_historical(bar_data, option_symbol, underlying_price, expiry, strike_price, delta, option_price)
                # print(f"Found short put at timestamp: {timestamp}")

            elif long_put_delta_range[0] <= delta <= long_put_delta_range[1]:
                long_put = create_option_dict_historical(bar_data, option_symbol, underlying_price, expiry, strike_price, delta, option_price)
                # print(f"Long put found at timestamp: {timestamp}")
            
            # Check spread width only when both options are found
            if short_put and long_put:
                current_spread_width = abs(short_put['strike_price'] - long_put['strike_price'])
                if not (spread_width[0] <= current_spread_width <= spread_width[1]):
                    print(f"Spread width of {spread_width} is outside the target range of ${spread_width[0]}-${spread_width[1]}; resetting search.")
                    # Reset both options to continue searching
                    short_put = None
                    long_put = None
                    continue
                else:
                    # Exit immediately when valid pair found
                    print(f"Valid spread found with width ${spread_width} at timestamp: {timestamp} for {short_put['symbol']} and {long_put['symbol']} at underlying price: {underlying_price}")
                    return short_put, long_put

    return short_put, long_put


def create_option_dict_historical(bar_data, symbol, underlying_price, expiry, strike_price, delta, option_price):
    """Create option dictionary from historical bar data"""
    return {
        'symbol': symbol,
        'strike_price': strike_price,
        'underlying_price': underlying_price,
        'expiration_date': expiry,
        'delta': delta,
        'option_price': option_price,
        'timestamp': bar_data['timestamp'],
        'volume': bar_data['volume'],
        'trade_count': bar_data['trade_count'],
        'vwap': bar_data['vwap'],
        'open': bar_data['open'],
        'high': bar_data['high'],
        'low': bar_data['low'],
        'close': bar_data['close']
    }

In [ ]:
short_put, long_put = find_short_and_long_puts(historical_stock_and_option_bars_data, RISK_FREE_RATE, SHORT_PUT_DELTA_RANGE, LONG_PUT_DELTA_RANGE)
short_put, long_put

## Executing a 0DTE Bull Put Spread Using Historical Stock and Option Bars

The `trade_0DTE_options_historical` function simulates trading a bull put vertical spread on options expiring the same day (0DTE) using historical market data.

### Strategy Overview
- **Sell a higher-strike put** (collect premium)
- **Buy a lower-strike put** (limit risk)  
- **Net result**: Receive credit upfront, profit if underlying stays above short strike

### How It Works
1. **Find Options**: Automatically selects short/long puts based on delta ranges and spread width ($2-$5)
2. **Monitor Position**: Tracks option prices and Greeks throughout the trading day
3. **Exit When**: One of four conditions is met (in priority order):

### Exit Conditions
| Condition | Trigger | Result |
|-----------|---------|---------|
| **Profit Target** | Spread price drops to target level | Take profit |
| **Delta Stop Loss** | Position delta exceeds risk threshold | Cut losses |
| **Assignment Risk** | Underlying drops below short strike | Avoid assignment |
| **Expiration** | End of trading day reached | Keep full credit |

### Key Assumptions
- No early assignment of OTM short put options
- Perfect liquidity (can exit at any time)
- Uses historical option "close" prices (no bid-ask spread)
- Single-day trading only (0DTE)

In [1]:
def trade_0DTE_options_historical(historical_stock_and_option_bars_data, risk_free_rate, delta_stop_loss_thres, target_stop_loss_percentage, short_put_delta_range, long_put_delta_range, spread_width):
    """
    Execute a 0DTE bull put vertical spread using historical data for backtesting.
    """

    short_put, long_put = find_short_and_long_puts(historical_stock_and_option_bars_data, risk_free_rate, short_put_delta_range, long_put_delta_range, spread_width)
    
    # Extract parameters
    short_symbol = short_put['symbol']
    long_symbol = long_put['symbol']
    short_strike = short_put['strike_price']
    long_strike = long_put['strike_price']
    expiration_date = short_put['expiration_date']
    entry_timestamp = short_put['timestamp']  # Get entry timestamp from option data
    short_price = short_put['option_price']
    long_price = long_put['option_price']
    
    # Calculate initial metrics
    credit_received = short_put['option_price'] - long_put['option_price']
    initial_total_delta = abs(short_put['delta']) - abs(long_put['delta'])
    delta_stop_loss = initial_total_delta * delta_stop_loss_thres
    target_profit_price = credit_received * target_stop_loss_percentage
    
    # Monitor through historical data starting after entry timestamp
    for timestamp in historical_stock_and_option_bars_data.keys():
        if timestamp <= entry_timestamp:
            continue  # Skip timestamps up to and including entry time
            
        option_bars = historical_stock_and_option_bars_data[timestamp]
        
        # Find current prices
        current_short_price = None
        current_long_price = None
        current_underlying_price = None
        
        
        for bar in option_bars:
            # If the the option symbol is same as short_symbol, then replace the current_short_price and current_underlying_price
            if bar['symbol'] == short_symbol:
                current_short_price = bar['close']
                current_underlying_price = bar.get('underlying_close')

            # If the the option symbol is same as long_symbol, then replace the current_long_price and current_underlying_price
            elif bar['symbol'] == long_symbol:
                current_long_price = bar['close']
                if current_underlying_price is None:
                    current_underlying_price = bar.get('underlying_close')
        
        # If any of the prices are not found, then skip the timestamp
        if not all([current_short_price, current_long_price, current_underlying_price]):
            # print(f"Prices were not found at timestamp: {timestamp}")
            continue
            
        current_spread_price = current_short_price - current_long_price
        
        # Calculate current deltas
        current_short_delta = calculate_delta_historical(current_short_price, short_strike, expiration_date, current_underlying_price, risk_free_rate, 'put', timestamp)
        current_long_delta = calculate_delta_historical(current_long_price, long_strike, expiration_date, current_underlying_price, risk_free_rate, 'put', timestamp)
        
        if current_short_delta is None or current_long_delta is None:
            # print(f"Delta was not calculated at timestamp: {timestamp}")
            continue
            
        current_total_delta = abs(current_short_delta) - abs(current_long_delta)
        
        # Check exit conditions (In live trading, we place the order to exit here)
        # If the spread price is less than the target profit price, then exit and return the profit
        if current_spread_price <= target_profit_price:
            return {
                'status': 'theoretical_profit', 
                'theoretical_pnl': (credit_received - current_spread_price) * 100, 
                'exit_time': timestamp,
                'short_put_symbol': short_symbol,
                'long_put_symbol': long_symbol,
                'entry_time': entry_timestamp
            }
        
        # If the total delta is greater than the delta stop loss, then exit and return the stop loss
        if current_total_delta >= delta_stop_loss:
            return {
                'status': 'stop_loss', 
                'theoretical_pnl': (credit_received - current_spread_price) * 100, 
                'exit_time': timestamp,
                'short_put_symbol': short_symbol,
                'long_put_symbol': long_symbol,
                'entry_time': entry_timestamp
            }
        
        # If the underlying price is less than the short strike price, then short put is being exercised and calculate the theoretical loss
        if current_underlying_price <= short_strike:
            theoretical_loss = short_price - long_price - short_strike + current_underlying_price
            return {
                'status': 'theoretical_loss', 
                'theoretical_pnl': theoretical_loss * 100, 
                'exit_time': timestamp,
                'short_put_symbol': short_symbol,
                'long_put_symbol': long_symbol,
                'entry_time': entry_timestamp
            }

    # Handle expiration
    final_timestamp = max(historical_stock_and_option_bars_data.keys())
    
    return {
        'status': 'expired', 
        'theoretical_pnl': credit_received, 
        'exit_time': final_timestamp,
        'short_put_symbol': short_symbol,
        'long_put_symbol': long_symbol,
        'entry_time': entry_timestamp
    }

## Running an Iterative Backtest

In [48]:
# Run iterative backtest
def run_iterative_backtest(max_iterations, underlying_symbol, start_date, end_date, buffer_pct, risk_free_rate, delta_stop_loss_thres, target_stop_loss_percentage, short_put_delta_range, long_put_delta_range, spread_width, time_interval=10):
    """
    Run iterative backtest that continuously finds and trades new option pairs throughout the day.
    """
    all_results = []
    
    try:
        # Get initial data
        stock_bars_data = get_daily_stock_bars_df(underlying_symbol, start_date, end_date)
        option_symbols_by_expiration = collect_option_symbols_by_expiration(stock_bars_data, underlying_symbol, buffer_pct)
        historical_stock_and_option_bars_data = get_stock_and_option_bars_data(option_symbols_by_expiration, underlying_symbol=underlying_symbol, time_interval=time_interval)
        
        # Initialize start time (first timestamp in data)
        current_start_time = min(historical_stock_and_option_bars_data.keys())
        
        iteration = 1
        
        while True:
            print(f"\n--- Iteration {iteration} ---")
            print(f"Starting from: {current_start_time}")
            
            # Filter data to only include timestamps after current_start_time
            filtered_historical_stock_and_option_bars_data = {
                timestamp: bars for timestamp, bars in historical_stock_and_option_bars_data.items() 
                if timestamp >= current_start_time
            }
            
            if not filtered_historical_stock_and_option_bars_data:
                print("No more data available. Ending iterations.")
                break
            
            # Execute trade (function will find options for bull put spread strategy internally)
            result = trade_0DTE_options_historical(filtered_historical_stock_and_option_bars_data, risk_free_rate, delta_stop_loss_thres, target_stop_loss_percentage, short_put_delta_range, long_put_delta_range, spread_width)
            
            if not result:
                print("Could not execute trade. Ending iterations.")
                break
            
            # Store result
            result['iteration'] = iteration
            all_results.append(result)
            
            print(f"Status: {result['status']} | theoretical PnL: ${result['pnl']:.2f} | {result['short_put_symbol']} & {result['long_put_symbol']} | Entry Time: {result['entry_time']} | Exit Time: {result['exit_time']}")
            
            # Update start time for next iteration to be after the exit time
            current_start_time = result['exit_time']
            
            # Add small buffer to ensure we don't include the exit timestamp
            current_start_time = current_start_time + pd.Timedelta(minutes=1)
            
            iteration += 1
            
            # Safety check to prevent infinite loops
            if iteration > max_iterations:  # Adjust as needed
                print("Maximum iterations reached. Stopping.")
                break
    
    except Exception as e:
        print(f"Error in iteration {iteration}: {e}")
    
    return all_results

In [ ]:
MAX_ITERATIONS = 100
BUFFER_PCT = 0.05
TIME_INTERVAL = 10
SPREAD_WIDTH = (2, 4)
start_date = datetime.now(NY_TZ).date() - timedelta(days=50)
end_date = datetime.now(NY_TZ).date() - timedelta(days=1)

# Execute iterative backtest
results = run_iterative_backtest(
    max_iterations=MAX_ITERATIONS, 
    underlying_symbol=underlying_symbol, 
    start_date=start_date, 
    end_date=end_date, 
    buffer_pct=BUFFER_PCT, 
    risk_free_rate=RISK_FREE_RATE, 
    delta_stop_loss_thres=DELTA_STOP_LOSS_THRES, 
    target_stop_loss_percentage=TARGET_STOP_LOSS_PERCENTAGE, 
    short_put_delta_range=SHORT_PUT_DELTA_RANGE, 
    long_put_delta_range=LONG_PUT_DELTA_RANGE, 
    spread_width=SPREAD_WIDTH,
    time_interval=TIME_INTERVAL
)

# Display summary
if results:
    total_pnl = sum(result['pnl'] for result in results)
    print(f"\n--- Summary ---")
    print(f"Total trades: {len(results)}")
    print(f"Total theoretical P&L: ${total_pnl:.2f}")
    for i, result in enumerate(results, 1):
        print(f"Trade {i}: {result['status']} | ${result['pnl']:.2f}")

In [ ]:
import matplotlib.pyplot as plt

# Convert to DataFrame and calculate cumulative P&L
df = pd.DataFrame(results)
df['cumulative_pnl'] = df['pnl'].cumsum()

# Convert entry_time to datetime and extract date
df['entry_time'] = pd.to_datetime(df['entry_time'])

# Create plot
plt.figure(figsize=(12, 6))
plt.plot(df['entry_time'], df['cumulative_pnl'], linewidth=2, marker='o', markersize=4)
plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)
plt.title('Cumulative P&L Over Time', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Cumulative P&L ($)')
plt.grid(True, alpha=0.3)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Add final P&L text
final_pnl = df['cumulative_pnl'].iloc[-1]
plt.text(df['entry_time'].iloc[int(len(df)*0.7)], final_pnl, f'Final P&L: ${final_pnl:.2f}', 
         fontsize=12, fontweight='bold', 
         bbox=dict(boxstyle="round,pad=0.3", facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

# Simple summary
print(f"\nTotal Trades: {len(df)}")
print(f"Total theoretical P&L: ${df['pnl'].sum():.2f}")
# print(f"Win Rate: {(df['pnl'] > 0).mean()*100:.1f}%")